<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/кейсы%20по%20анализу%20данных/Работа%20с%20датами%2002.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

Работа с датами — неотъемлемая часть решения многих аналитических задач. Некоторые инструменты, например, Excel, имеют встроенные инструменты по распознаванию дат в автоматическом режиме. Но не всегда даже эти инструменты дают корректный результат при работе с базами данных.

Например, запись `01.12.17`, скорее всего, будет распознана как `1 декабря 2017 года`. Однако в американском стандарте записи дат это `12 января 2017 года`.

Также многие выгрузки из систем и баз данных имеют свой служебный формат. Например, формат времени из разных систем может отличаться:

* `2018-11-09 15:45:21`
* `11/09/2018 3:45:20 PM`
* `2018-11-09T15:45:21.2984`

Для всех этих случаев необходимо задавать формат распознавания дат, уметь сравнивать их между собой. Также часто необходимо корректно прибавлять к датам разные временные интервалы. Например, час или день.

Представим, что нам надо посчитать показатели рекламной кампании за определенный период. Например, для человека удобно дату начала и конца выгрузки задавать в формате строки:

In [0]:
startDate = '2017-12-01'
endDate = '2017-12-31'

Однако сейчас переменные `startDate` и `endDate` — просто строки, которые нельзя преобразовывать, как даты. Для этого необходимо сначала перевести их в специальный формат.

### БИБЛИОТЕКА DATETIME

Будем работать с файлом [`data.tsv`](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/data.tsv). В нем есть столбец `date`. 

Возьмем для примера первое значение в этом столбце:

In [0]:
date_string = '05.10.2016  23:18'

Сейчас переменная `date_string` является просто строкой.

In [0]:
type(date_string)

str

Соответственно, сейчас мы не можем выполнять со строкой никаких операций. Например, прибавить к этой дате неделю или посчитать, к какому году она относится. Нам необходимо перевести эту дату в формат `datetime`. Для этого в Python есть библиотека с аналогичным названием `datetime`. 

Импортируем ее в наш скрипт:

In [0]:
import datetime

На текущем занятии мы будем использовать из этой библиотеки следующие модули: `datetime` для распознавания формата дат и `timedelta` для прибавления к текущей дате определенный временной интервал. Нам придется вызывать эти модули следующим образом: `datetime.datetime` и `datetime.timedelta`, что достаточно громоздко. Чтобы сделать код более читабельным, давайте импортируем эти модули следующим образом:

In [0]:
from datetime import datetime, timedelta

### ФОРМАТЫ ДАТЫ И ВРЕМЕНИ

Чтобы перевести переменную `date_string` в формат даты и времени, необходимо указать, в каком формате записана наша переменная. Допустим, что в этом примере цифры значат следующее:

* 05 - день
* 10 - месяц
* 2016 - год
* 23 - часы
* 18 - минуты

В Python для каждого формата даты и времени есть свое обозначение. Например, чтобы указать формат `'часы в формате от 0 до 24'`, надо на [странице](https://docs.python.org/3/library/datetime.html) в таблице форматов (она в конце страницы) найти соответствующее обозначение — `'Hour (24-hour clock) as a zero-padded decimal number'`, т.е. `%H`.

В этой таблице есть много форматов, даже на случаи обозначения дней недели, как `Sun`, `Mon` (первая строка таблицы) и использование `AM` и `PM`.

Давайте расшифруем формат переменной `date_string`. Для этого используется метод `strptime`, который переводит переменную типа строка в переменную типа дата и время.

В качестве первого аргумента ставим строковую переменную (`date_string`), потом указываем формат (при указании формата обязательно учитываем все точки и двоеточия):

In [0]:
date_string = '05.10.2016  23:18'
datetime.strptime(date_string, '%d.%m.%Y %H:%M')

datetime.datetime(2016, 10, 5, 23, 18)

Вывод `datetime.datetime(2016, 10, 5, 23, 18)` означает, что мы верно расшифровали формат. Если бы мы ошиблись, то получили бы ошибку:

In [0]:
#datetime.strptime(date_string, '%Y %H:%M')

Запишем наш результат в переменную `date_datetime` и посмотрим, что с ней можно делать:

In [0]:
date_datetime = datetime.strptime(date_string, '%d.%m.%Y %H:%M')
type(date_datetime)

datetime.datetime

Теперь можем получать множество характеристик даты:

In [0]:
date_datetime.year # если хотим сгруппировать статистику покупок по году

2016

In [0]:
date_datetime.hour # если строим отчет активности покупок по часам

23

### Задание 1

С помощью метода `datetime.strptime` переведите строку `'May 9 2017 9:00AM'` в формат `datetime`.

Выделите и запишите номер часа в этой дате (в виде целого числа). Подсказка: у обозначения месяца `May` формат `%b`, у `AM` - `%p`

In [0]:
date = 'May 9 2017 9:00AM'
date_datetime = datetime.strptime(date, '%b %d %Y %I:%M%p')
date_datetime.hour

9

### ПРИБАВЛЕНИЕ ДНЕЙ

Необходимость прибавлять временной интервал к дате очень часто встречается в самых разнообразных задачах. Например, при заборе данных из внешних источников с помощью API для каждого дня отдельно (для увеличения точности выгрузки).

Допустим, у нас есть дата начала выгрузки `startDate`. Необходимо прибавить к этой дате день. Воспользуемся модулем `timedelta`.

In [0]:
startDate = '2017-01-01'

Переводим строковую переменную `startDate` в формат `datetime`:

In [0]:
startDate_datetime = datetime.strptime(startDate, '%Y-%m-%d')

Теперь можем прибавлять к ней нужные временные интервалы с помощью `timedelta`:

In [0]:
startDate_datetime + timedelta(days=1)

datetime.datetime(2017, 1, 2, 0, 0)

В результате получили `2 января 2017 года`. Посмотрим, что получится, если отнять от `startDate_datetime (от 1 января) 7 дней`:

In [0]:
startDate_datetime + timedelta(days=-7)

datetime.datetime(2016, 12, 25, 0, 0)

### ПРИБАВЛЕНИЕ ВРЕМЕНИ

Для увеличения даты и времени на определенное количество часов или минут просто меняем название параметра. Попробуем добавить к дате выгрузки 1 час:

In [0]:
startDate_datetime += timedelta(hours=1)
startDate_datetime

datetime.datetime(2017, 1, 1, 1, 0)

### Задание 2

Возьмите дату из прошлого шага `'May 9 2017 9:00AM'`, переведите в формат `datetime` и прибавьте к ней час с помощью `timedelta`.

Какой результат будет на экране в формате `datetime`? Ответ должен иметь вид `datetime.datetime(...)`

In [0]:
date_datetime += timedelta(hours=1)
date_datetime

datetime.datetime(2017, 5, 9, 10, 0)

### ПЕРЕВОД DATETIME В СТРОКУ

После вычислений с датами можно легко вернуть результат обратно в строку. Для этого воспользуемся методом `strftime`, указав в качестве аргумента желаемый формат:

In [0]:
startDate_datetime.strftime( '%Y-%m-%d %H:%M:%S' )

'2017-01-01 01:00:00'

### Задание 3

Возьмите результат прошлого упражнения (прибавление часа к 9 мая) и запишите результат в формате `'%Y-%m-%d'`. Какой будет результат?

In [0]:
str_date = date_datetime.strftime('%Y-%m-%d')
str_date

'2017-05-09'

### ПОСТАНОВКА ЗАДАЧИ

После того, как мы освоили работу с отдельными датами, давайте посмотрим, как работать с большим количеством дат. 

Для этого мы реализуем алгоритм, который по дате начала и конца выгрузки будет «пробегать» все значения по дням. Этот метод часто необходим при получении данных от внешних систем, а также обработки больших выгрузок частями. Например, по дням или по часам. Типичный пример — выгрузка сложных отчетов из Google Analytics по дням, чтобы уменьшить сэмплирование данных.

В дальнейшем такой навык нам понадобится для определения покупательской активности по часам и дням или любой другой аналитики в больших файлах.

### ПОДГОТОВКА ПЕРЕМЕННЫХ

Потренируемся на простом примере. Представим, что нам необходимо из огромной таблицы с данными за год выяснить показатели за новогодние праздники с 1 по 7 января, и научимся в этом шаге перебирать необходимые даты. 

Начальную и конечную дату будем задавать в удобном для человека строковом виде:

In [0]:
startDate = '2017-01-01'
endDate = '2017-01-07'

Поскольку нам надо в цикле работать с датами, то переведем строковые переменные в формат `datetime`:

In [0]:
startDate_datetime = datetime.strptime(startDate, '%Y-%m-%d')
endDate_datetime = datetime.strptime(endDate, '%Y-%m-%d')
print(startDate_datetime, endDate_datetime)

2017-01-01 00:00:00 2017-01-07 00:00:00


### СОЗДАЕМ ЦИКЛ

Заведем переменную `current_day`, которая в цикле будет изменяться от 1 до 7 января и будет иметь тип `datetime`. В первом шаге цикла эта переменная будет равна 1 января:

In [0]:
current_day = startDate_datetime

Теперь в цикле будем увеличивать значение `current_day` на 1 день, пока ее значение не превысит дату конца выгрузки. Для составления таких циклов используется конструкция `while`. Этот цикл будет работать, пока указанное в нем условие выполняется. Простой пример для вывода нескольких чисел:

In [0]:
a = 1
while a < 5:
    print(a)    
    a += 1

1
2
3
4


Напишем точно такой же цикл, только заменим `a` на `current_day`, а число 5 на 7 января:

In [0]:
current_day = startDate_datetime
while current_day <= endDate_datetime:
    print(current_day)    
    current_day += timedelta(days=1)

2017-01-01 00:00:00
2017-01-02 00:00:00
2017-01-03 00:00:00
2017-01-04 00:00:00
2017-01-05 00:00:00
2017-01-06 00:00:00
2017-01-07 00:00:00


Для удобства вывода заменим вывод `current_day` на более наглядный строковый эквивалент в формате даты:

In [0]:
current_day = startDate_datetime
while current_day <= endDate_datetime:
    print(current_day.strftime('%Y-%m-%d'))
    current_day += timedelta(days=1)

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07


Итак, мы получили цикл, в котором переменная `current_day` «пробегает» между 1 и 7 января по дням.

### Задание 4

Напишите алгоритм, который «пробегает» период 1 до 3 января включительно по часам. Формат вывода `'%Y-%m-%d %H:%M:%S'`.

Какое значение будет последним для 2 января (т. е. 23 часа 2 января)? Результат должен иметь формат `%Y-%m-%d %H:%M:%S`

In [0]:
startDate_str = '2020-1-1 00:00:00'
endDate_str = '2020-1-3 23:00:00'

In [0]:
startDate_datetime = datetime.strptime(startDate_str, '%Y-%m-%d %H:%M:%S')
endDate_datetime = datetime.strptime(endDate_str, '%Y-%m-%d %H:%M:%S')

In [0]:
current_day = startDate_datetime
dates_list = []
while current_day <= endDate_datetime:
  dates_list.append(current_day.strftime('%Y-%m-%d %H:%M:%S'))
  current_day += timedelta(hours=1)

In [0]:
dates_list # ->  Ответ: '2020-01-02 23:00:00'

['2020-01-01 00:00:00',
 '2020-01-01 01:00:00',
 '2020-01-01 02:00:00',
 '2020-01-01 03:00:00',
 '2020-01-01 04:00:00',
 '2020-01-01 05:00:00',
 '2020-01-01 06:00:00',
 '2020-01-01 07:00:00',
 '2020-01-01 08:00:00',
 '2020-01-01 09:00:00',
 '2020-01-01 10:00:00',
 '2020-01-01 11:00:00',
 '2020-01-01 12:00:00',
 '2020-01-01 13:00:00',
 '2020-01-01 14:00:00',
 '2020-01-01 15:00:00',
 '2020-01-01 16:00:00',
 '2020-01-01 17:00:00',
 '2020-01-01 18:00:00',
 '2020-01-01 19:00:00',
 '2020-01-01 20:00:00',
 '2020-01-01 21:00:00',
 '2020-01-01 22:00:00',
 '2020-01-01 23:00:00',
 '2020-01-02 00:00:00',
 '2020-01-02 01:00:00',
 '2020-01-02 02:00:00',
 '2020-01-02 03:00:00',
 '2020-01-02 04:00:00',
 '2020-01-02 05:00:00',
 '2020-01-02 06:00:00',
 '2020-01-02 07:00:00',
 '2020-01-02 08:00:00',
 '2020-01-02 09:00:00',
 '2020-01-02 10:00:00',
 '2020-01-02 11:00:00',
 '2020-01-02 12:00:00',
 '2020-01-02 13:00:00',
 '2020-01-02 14:00:00',
 '2020-01-02 15:00:00',
 '2020-01-02 16:00:00',
 '2020-01-02 17:

### ФУНКЦИЯ ДЛЯ РАБОТЫ

Далее функция, которая сразу формирует список с датами в заданном диапазоне. Вы можете скопировать ее и использовать в работе. Пример работы функции приведен в ее описании: 

In [0]:
def date_range(start_date, end_date):
    """
    Возвращает список дат между start_date и end_date с шагом в день.
    Если start_date > end_date, то возвращает пустой список. 
    Пример
    date_range('2018-01-01', '2018-01-07')
    [
        '2018-01-01',
        '2018-01-02',
        '2018-01-03',
        '2018-01-04',
        '2018-01-05',
        '2018-01-06',
        '2018-01-07'
    ]
    """ 
    date_range_list = []
    current_date = start_date  
    current_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')   
    while current_date_dt <= end_date_dt:
        date_range_list.append(current_date)        
        current_date_dt += timedelta(days=1)
        current_date = current_date_dt.strftime('%Y-%m-%d')    
    return date_range_list

### ПОСТАНОВКА ЗАДАЧИ

Нам необходимо для пользователей из датасета [`data.tsv`](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/data.tsv) посчитать метрику `lifetime`, т. е. среднее время жизни (с точки зрения покупок, конечно). Т. е. если пользователь сделал `2` и более заказа, то для него необходимо посчитать разницу во времени между первой и последней покупкой и, соответственно, усреднить эти значения для всех пользователей.

Основной проблемой будет то, что время покупки у нас задано в виде строки. Соответственно, необходимо перевести эту дату в числа и считать разницу между первой и последней покупкой.

### ПЕРЕВОДИМ СТРОКИ В ДАТЫ
Итак, посмотрим, как выглядят наши данные:

In [0]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/data.tsv', sep='\t')
data.head()5

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8


Переведем столбец `date` в тип `datetime` с помощью функции:

In [0]:
def convert_to_datetime(row):
    return datetime.strptime(row['date'], '%d.%m.%Y %H:%M')

Проверяем результат:

In [0]:
data['datetime'] = data.apply(convert_to_datetime, axis=1)
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid,datetime
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2,2016-10-05 23:18:00
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6,2016-10-09 21:40:00
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2,2016-10-05 03:23:00
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8,2016-10-05 23:19:00
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8,2016-10-01 04:57:00


### ФОРМАТ UNIXTIME

Для подсчета разницы в датах есть много методов. Одним из самых простых является перевод времени в формат `unixtime`. Он означает количество секунд, прошедшее с `1 января 1970 года`. Этот формат очень популярен и используется во многих системах. Попробуем использовать его для нашей задачи.

Для перевод столбца `datetime` в unix-формат даты используем библиотеку `time` и следующую функцию:

In [0]:
import time
def make_unix_time(row):
    return time.mktime(row['datetime'].timetuple())

Заведем столбец `unixtime` и посмотрим, что получилось:

In [0]:
data['unixtime'] = data.apply(make_unix_time, axis=1)
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid,datetime,unixtime
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2,2016-10-05 23:18:00,1.475709e+09
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6,2016-10-09 21:40:00,1.476049e+09
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2,2016-10-05 03:23:00,1.475638e+09
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8,2016-10-05 23:19:00,1.475710e+09
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8,2016-10-01 04:57:00,1.475298e+09


Дальнейшие действия по расчету `lifetime` будут составлять самостоятельное задание.

### ЗАДАНИЕ
Для завершения подсчета `lifetime` вам необходимо сделать следующее:

1. Сгруппировать датафрейм data по столбцу user_id, посчитав для столбца unixtime максимальное и минимальное значение для каждого пользователя.
2. Посчитать столбец diff с разностью максимального и минимального значения столбца unixtime из пункта 1.
3. Исключить из расчета пользователей, у которых разница diff равна 0.
4. Посчитать среднее значение столбца diff после фильтрации.
5. Вы получите ответ в секундах. Пересчитайте его в количество дней.

Каково значение `lifetime` в днях? Результат округлите до первого знака после запятой.

In [0]:
# Пункт 1
dataMinMax = data.groupby(['user_id'])['unixtime'].agg(['min','max'])
dataMinMax

,min,max
user_id,,
1010,1.475709e+09,1.475709e+09
1036,1.476049e+09,1.476049e+09
1041,1.475638e+09,1.475710e+09
1042,1.475298e+09,1.475945e+09
1047,1.475329e+09,1.475704e+09
...,...,...
1393,1.475732e+09,1.475732e+09
1394,1.475625e+09,1.475625e+09
1397,1.475644e+09,1.475644e+09


In [0]:
def sub_min_max(row):
  return row['max'] - row['min']

In [0]:
# Пункт 2
dataMinMax['diff'] = dataMinMax.apply(sub_min_max, axis = 1)
dataMinMax

,min,max,diff
user_id,,,
1010,1.475709e+09,1.475709e+09,0.0
1036,1.476049e+09,1.476049e+09,0.0
1041,1.475638e+09,1.475710e+09,71760.0
1042,1.475298e+09,1.475945e+09,647340.0
1047,1.475329e+09,1.475704e+09,374520.0
...,...,...,...
1393,1.475732e+09,1.475732e+09,0.0
1394,1.475625e+09,1.475625e+09,0.0
1397,1.475644e+09,1.475644e+09,0.0


In [0]:
# Пункт 3 
dataMinMax = dataMinMax[dataMinMax['diff'] != 0]
dataMinMax

,min,max,diff
user_id,,,
1041,1.475638e+09,1.475710e+09,71760.0
1042,1.475298e+09,1.475945e+09,647340.0
1047,1.475329e+09,1.475704e+09,374520.0
1052,1.475834e+09,1.475955e+09,121020.0
1057,1.475580e+09,1.475981e+09,401700.0
1100,1.475331e+09,1.475940e+09,608340.0
1108,1.475296e+09,1.476002e+09,705780.0
1112,1.475645e+09,1.475971e+09,326280.0
1113,1.475749e+09,1.476015e+09,266400.0


In [0]:
# Пункт 4
meanDiff = dataMinMax['diff'].mean()
meanDiff

301143.21428571426

In [0]:
# Пункт 5
round(meanDiff/(3600*24),1)

3.5